# Comparison Using LFW Dataset - VGGFace (Resnet50)

Several feature extractor:
1. Face Embedding: Facenet
2. Face Embedding: VGG Face
3. Face Embedding: VGG Face - VGG16
4. Face Embedding: VGG Face - RESNET50
5. LBPH (Local Binary Pattern Histogram)

In [1]:
from scipy.spatial.distance import euclidean
from sklearn.metrics import accuracy_score, f1_score
from keras import backend as K
from feature_extractor.face_feature_extractor import FaceFeatureExtractor
import numpy as np
import cv2 as cv
from mtcnn.mtcnn import MTCNN
import matplotlib.pyplot as plt

/home/agusgun/anaconda3/envs/basic/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Read Dataset

In [2]:
import os

In [3]:
DIR_PATH = '../lfw/'
image_path_list = []
labels = []
name_dictionary = {}
counter = 0
for root, dirs, files in os.walk(DIR_PATH):
    for filename in files:
        person_name = ' '.join(filename.split('.')[0].split('_')[0:-1]) 
        file_path = os.path.join(root, filename)
        if person_name not in name_dictionary:
            counter += 1
            name_dictionary[person_name] = counter
        image_path_list.append(file_path)
        labels.append(name_dictionary[person_name])

In [4]:
print(len(labels))
print(len(image_path_list))

13233
13233


In [5]:
image_path_list[0:5]

['../lfw/Ryan_Newman/Ryan_Newman_0001.jpg',
 '../lfw/Dimitar_Berbatov/Dimitar_Berbatov_0001.jpg',
 '../lfw/Ed_Rendell/Ed_Rendell_0001.jpg',
 '../lfw/Joe_Crede/Joe_Crede_0001.jpg',
 '../lfw/Norman_Mailer/Norman_Mailer_0001.jpg']

## Shuffle It

In [6]:
import random

In [7]:
temp = list(zip(image_path_list, labels))
random.Random(0).shuffle(temp) # custom seed
image_path_list, labels = zip(*temp)

In [8]:
image_path_list[0:5]

('../lfw/Tommy_Maddox/Tommy_Maddox_0001.jpg',
 '../lfw/David_Millar/David_Millar_0001.jpg',
 '../lfw/Gregg_Popovich/Gregg_Popovich_0004.jpg',
 '../lfw/Shimon_Peres/Shimon_Peres_0001.jpg',
 '../lfw/Rudolph_Giuliani/Rudolph_Giuliani_0024.jpg')

In [9]:
labels[0:5]

(3038, 1517, 3115, 834, 1016)

In [10]:
name_dictionary['John Ashcroft']

482

# Experiment

In [11]:
FACENET_MODEL_PATH = 'model_data/facenet/20180402-114759'
VGGFACE_MODEL_PATH = 'model_data/vgg_face_weights.h5'

In [12]:
mtcnn_detector = MTCNN()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [13]:
def predict(image_representation_list, true_label, test_representation, min_distance):
    minimum_label = None
    minimum_distance = min_distance
    
    for idx, image_representation in enumerate(image_representation_list):
        distance = euclidean(image_representation, test_representation)
        if distance < minimum_distance:
            minimum_distance = distance
            minimum_label = true_label[idx]
    
    return minimum_label

In [14]:
def compute_score(predictions, labels):
    count_same = 0
    for idx, prediction in enumerate(predictions):
        if labels[idx] == prediction:
            count_same += 1
    return count_same / len(predictions)

## VGGFace - Resnet50

Based on experiment VGGFace - Resnet50 have distance below 100 for euclidean if two image are the same person

In [15]:
feature_extractor = FaceFeatureExtractor(None, extractor_name='vgg_face_resnet50')

### Testing Starts Here

In [ ]:
THRESHOLD = 100

In [16]:
image_representation_database = []
image_representation_labels = []
prediction_result = []

In [17]:
%%time
for idx, image_path in enumerate(image_path_list):
    if idx % 1000 == 0:
        print("Checkpoint", idx)
    img = cv.cvtColor(cv.imread(image_path), cv.COLOR_BGR2RGB)
    detection_result = mtcnn_detector.detect_faces(img)
    cropped_image = None
    for face in detection_result:
        face_bbox = face['box']
        x, y, w, h = face_bbox
        if x < 0:
            x = 0
        if y < 0:
            y = 0
        cropped_image = img[y:y+h, x:x+w]
        break
    
    if not cropped_image is None:
        feature_test = feature_extractor.extract_image(cropped_image)
        prediction = predict(image_representation_database, image_representation_labels, feature_test, THRESHOLD)
        if prediction == None:
            label = labels[idx]
            if label not in image_representation_labels:
                image_representation_labels.append(label)
                image_representation_database.append(feature_test)
                prediction_result.append(label)
            else: # false prediction
                prediction_result.append(-1) 
        else:
            prediction_result.append(prediction)
    else: # failed to detect faces
        prediction_result.append(-1)

Checkpoint 0
Checkpoint 1000
Checkpoint 2000
Checkpoint 3000
Checkpoint 4000
Checkpoint 5000
Checkpoint 6000
Checkpoint 7000
Checkpoint 8000
Checkpoint 9000
Checkpoint 10000
Checkpoint 11000
Checkpoint 12000
Checkpoint 13000
CPU times: user 3h 55min 7s, sys: 1h 55min 24s, total: 5h 50min 32s
Wall time: 1h 12min 46s


In [18]:
feature_extractor.extract_image(img)

array([0.        , 0.        , 0.        , ..., 0.06914644, 0.        ,
       0.17514291], dtype=float32)

In [19]:
image_path_list[37]

'../lfw/George_W_Bush/George_W_Bush_0233.jpg'

In [20]:
compute_score(prediction_result, labels)

0.7877276505705434

In [21]:
len(prediction_result)

13233

In [22]:
len(labels)

13233

In [23]:
# feature_extractor.extractor.close_session()

In [24]:
image_path_list = sorted(image_path_list)

In [25]:
image_path_list[31:50]

['../lfw/Abdullah_Gul/Abdullah_Gul_0001.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0002.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0003.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0004.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0005.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0006.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0007.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0008.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0009.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0010.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0011.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0012.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0013.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0014.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0015.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0016.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0017.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0018.jpg',
 '../lfw/Abdullah_Gul/Abdullah_Gul_0019.jpg']

In [26]:
%%time
for idx, image_path in enumerate(image_path_list[31:32]):
    if idx % 1000 == 0:
        print("Checkpoint", idx)
    img = cv.cvtColor(cv.imread(image_path), cv.COLOR_BGR2RGB)
    detection_result = mtcnn_detector.detect_faces(img)
    cropped_image = None
    for face in detection_result:
        face_bbox = face['box']
        x, y, w, h = face_bbox
        if x < 0:
            x = 0
        if y < 0:
            y = 0
        cropped_image = img[y:y+h, x:x+w]
        break
    
    if not cropped_image is None:
        feature_test = feature_extractor.extract_image(cropped_image)
        prediction = predict(image_representation_database, image_representation_labels, feature_test, THRESHOLD)
        if prediction == None:
            label = labels[idx]
            if label not in image_representation_labels:
                image_representation_labels.append(label)
                image_representation_database.append(feature_test)
                prediction_result.append(label)
            else: # false prediction
                prediction_result.append(-1) 
        else:
            prediction_result.append(prediction)
    else: # failed to detect faces
        prediction_result.append(-1)

Checkpoint 0
CPU times: user 1.19 s, sys: 623 ms, total: 1.81 s
Wall time: 1.24 s


In [27]:
base_feature = feature_test.copy()
base_feature

array([0.        , 0.64275444, 0.16716784, ..., 0.01909242, 2.3434634 ,
       0.49065006], dtype=float32)

In [28]:
%%time
for idx, image_path in enumerate(image_path_list[31:50]):
    if idx % 1000 == 0:
        print("Checkpoint", idx)
    img = cv.cvtColor(cv.imread(image_path), cv.COLOR_BGR2RGB)
    detection_result = mtcnn_detector.detect_faces(img)
    cropped_image = None
    for face in detection_result:
        face_bbox = face['box']
        x, y, w, h = face_bbox
        if x < 0:
            x = 0
        if y < 0:
            y = 0
        cropped_image = img[y:y+h, x:x+w]
        break
    
    if not cropped_image is None:
        feature_test = feature_extractor.extract_image(cropped_image)
        distance = euclidean(feature_test, base_feature)
        print(distance)

Checkpoint 0
0.0
82.52213287353516
84.78237915039062
78.82715606689453
79.36217498779297
60.43344497680664
48.81595993041992
69.8621826171875
106.51448822021484
67.05977630615234
58.84721755981445
52.67277526855469
52.3817253112793
73.55290222167969
65.49700164794922
78.40837097167969
60.621822357177734
72.69193267822266
67.33021545410156
CPU times: user 22 s, sys: 12.2 s, total: 34.1 s
Wall time: 12.9 s


In [29]:
%%time
for idx, image_path in enumerate(image_path_list[0:30]):
    if idx % 1000 == 0:
        print("Checkpoint", idx)
    img = cv.cvtColor(cv.imread(image_path), cv.COLOR_BGR2RGB)
    detection_result = mtcnn_detector.detect_faces(img)
    cropped_image = None
    for face in detection_result:
        face_bbox = face['box']
        x, y, w, h = face_bbox
        if x < 0:
            x = 0
        if y < 0:
            y = 0
        cropped_image = img[y:y+h, x:x+w]
        break
    
    if not cropped_image is None:
        feature_test = feature_extractor.extract_image(cropped_image)
        distance = euclidean(feature_test, base_feature)
        print(distance)

Checkpoint 0
136.31793212890625
138.59767150878906
136.0935516357422
139.9056396484375
126.19060516357422
129.37594604492188
135.495361328125
131.88719177246094
142.462646484375
128.8269805908203
133.00912475585938
121.98558807373047
130.2308807373047
130.763916015625
130.13465881347656
126.12809753417969
141.56590270996094
133.5976104736328
142.20700073242188
121.29850006103516
127.96356201171875
123.25005340576172
134.73443603515625
125.16188049316406
132.94863891601562
122.6933822631836
114.6954574584961
137.6082763671875
115.60914611816406
122.6716537475586
CPU times: user 32.7 s, sys: 16.8 s, total: 49.5 s
Wall time: 12.3 s
